<a href="https://colab.research.google.com/github/akukudala/data603-sp22/blob/main/Homework/class03/MapReduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mrjob

     |████████████████████████████████| 439 kB 3.2 MB/s 


In [2]:
%%file wc.py
from mrjob.job import MRJob

class MRWordFrequencyCount(MRJob):

    def mapper(self, _, line):
        yield "chars", len(line)
        yield "words", len(line.split())
        yield "lines", 1

    def reducer(self, key, values):
        yield key, sum(values)


if __name__ == '__main__':
    MRWordFrequencyCount.run()

Writing wc.py


In [3]:
!curl https://www.gutenberg.org/files/2600/2600-0.txt -o warpeace.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3280k  100 3280k    0     0  7226k      0 --:--:-- --:--:-- --:--:-- 7210k


In [4]:
import wc

mr_job = wc.MRWordFrequencyCount(args=['warpeace.txt'])
with mr_job.make_runner() as runner:
    runner.run()
    for key, value in mr_job.parse_output(runner.cat_output()):
        print(key, value)

No configs specified for inline runner


words 566334
chars 3161491
lines 66032


In [5]:
!curl https://raw.githubusercontent.com/PacktPublishing/Frank-Kanes-Taming-Big-Data-with-Apache-Spark-and-Python/master/1800.csv -o 1800.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 62728  100 62728    0     0   297k      0 --:--:-- --:--:-- --:--:--  297k


In [6]:
!tail -n 10 1800.csv

ITE00100554,18001230,TMAX,50,,,E,
ITE00100554,18001230,TMIN,31,,,E,
GM000010962,18001230,PRCP,13,,,E,
EZE00100082,18001230,TMAX,96,,,E,
EZE00100082,18001230,TMIN,14,,,E,
ITE00100554,18001231,TMAX,50,,,E,
ITE00100554,18001231,TMIN,25,,,E,
GM000010962,18001231,PRCP,16,,,E,
EZE00100082,18001231,TMAX,14,,,E,
EZE00100082,18001231,TMIN,-4,,,E,


In [7]:
%%file max_temp.py
from mrjob.job import MRJob

class MRMaxTemperature(MRJob):
    
    @staticmethod
    def to_fahrenheit(cels):
        celsius = float(cels) / 10.0
        fahrenheit = celsius * 1.8 + 32.0
        return fahrenheit

    def mapper(self, _, line):
        (location, date, type, data, x, y, z, w) = line.split(',')
        if (type == 'TMAX'):
            temperature = self.to_fahrenheit(data)
            yield location, temperature

    def reducer(self, location, temps):
        yield location, max(temps)


if __name__ == '__main__':
    MRMaxTemperature.run()
    

Writing max_temp.py


In [8]:
import max_temp
mr_job = max_temp.MRMaxTemperature(args=['1800.csv'])
with mr_job.make_runner() as runner:
    runner.run()
    for key, value in mr_job.parse_output(runner.cat_output()):
        print(key, value)

No configs specified for inline runner


EZE00100082 90.13999999999999
ITE00100554 90.13999999999999
